# 🦁 PyKwant Tutorial: Portfolio Optimization

This notebook introduces `pykwant.optimization`, a module for constructing optimal portfolios.

Modern Portfolio Theory (Markowitz) seeks to find the best trade-off between Risk (Volatility) and Return.
Instead of using heavy quadratic programming solvers, **PyKwant** uses a robust **Monte Carlo** approach:
1. We generate thousands of random portfolios.
2. We calculate Return and Volatility for each.
3. We filter for the best ones (Efficient Frontier).

## 1. Setup Market Assumptions

Let's define a universe of 3 assets with different characteristics:
* **Asset A (Bonds)**: Low Risk, Low Return.
* **Asset B (Equities)**: High Risk, High Return.
* **Asset C (Gold)**: Low Correlation, Medium Risk.

In [1]:
from pykwant import optimization

# 1. Expected Annual Returns
# Bonds: 3%, Equities: 10%, Gold: 5%
expected_returns = [0.03, 0.10, 0.05]

# 2. Covariance Matrix (Risk & Correlation)
# Diagonal = Variance (Vol^2)
# Off-diagonal = Covariance

# Volatilities assumption:
# Bonds: 5%  -> Var = 0.0025
# Stocks: 20% -> Var = 0.0400
# Gold: 15%   -> Var = 0.0225

# Correlations assumption:
# Bonds/Stocks = 0.2
# Bonds/Gold = 0.0
# Stocks/Gold = 0.1

cov_matrix = [
    [0.0025, 0.0002, 0.0000],  # Bonds row
    [0.0002, 0.0400, 0.0030],  # Stocks row
    [0.0000, 0.0030, 0.0225],  # Gold row
]

asset_names = ["Bonds", "Stocks", "Gold"]

## 2. Run Monte Carlo Optimization

We simulate **50,000** random portfolios to explore the feasible region.

In [2]:
# Risk Free Rate for Sharpe Ratio calculation
rf_rate = 0.02

max_sharpe, min_vol = optimization.optimize_portfolio_monte_carlo(
    expected_returns, cov_matrix, num_portfolios=50_000, risk_free_rate=rf_rate, seed=42
)

print("Optimization Complete.")

Optimization Complete.


## 3. Analyze: Minimum Volatility Portfolio

This portfolio sits at the leftmost tip of the Efficient Frontier. It is purely defensive.
We expect a high allocation to Bonds (lowest variance) and some diversification into Gold.

In [3]:
def print_portfolio(title, stats):
    print(f"--- {title} ---")
    print(f"Expected Return: {stats.expected_return:.2%}")
    print(f"Volatility:      {stats.volatility:.2%}")
    print(f"Sharpe Ratio:    {stats.sharpe_ratio:.2f}")
    print("\nAllocation:")
    for name, weight in zip(asset_names, stats.weights, strict=True):
        print(f"{name:<10}: {weight:>6.1%}")


print_portfolio("Minimum Volatility Portfolio", min_vol)

--- Minimum Volatility Portfolio ---
Expected Return: 3.49%
Volatility:      4.66%
Sharpe Ratio:    0.32

Allocation:
Bonds     :  86.6%
Stocks    :   4.4%
Gold      :   9.0%


## 4. Analyze: Maximum Sharpe Ratio Portfolio

This is the "Tangency Portfolio". It offers the highest premium per unit of risk. 
It typically mixes assets to capture growth (Stocks) while dampening volatility via diversification.

In [4]:
print_portfolio("Max Sharpe Ratio Portfolio", max_sharpe)

--- Max Sharpe Ratio Portfolio ---
Expected Return: 5.29%
Volatility:      6.97%
Sharpe Ratio:    0.47

Allocation:
Bonds     :  56.1%
Stocks    :  28.2%
Gold      :  15.7%


## 5. Interpretation

By using `pykwant`, we found the optimal weights without needing `scipy.optimize` or installing external solvers.

* **Min Volatility**: Heavily weighted towards Bonds, as expected.
* **Max Sharpe**: A balanced mix (e.g., 40/40/20) that captures the high return of stocks and the diversification benefit of gold to smooth out the ride.